# Project IART - 2023

In [2]:
import pandas
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

from IPython.display import Image
from subprocess import call

from sklearn.exceptions import ConvergenceWarning

from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

import sklearn.tree as tree  # 1. Decision Tree
from sklearn.neural_network import MLPClassifier  # 2. Neural Network
from sklearn.neighbors import KNeighborsClassifier # 3. KNN (K-Nearest Neighbors)
from sklearn.svm import SVC  # 4. Support Vector Machine (SVM)
from sklearn.ensemble import RandomForestClassifier  # 5. Random Forest
from sklearn.linear_model import LogisticRegression  # 6. Logistic Regression


from sklearn.metrics import confusion_matrix as conf_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# visualization
from sklearn.manifold import TSNE


from sklearn.mixture import GaussianMixture
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# data
import seaborn as sns

modelResults = {}


In [3]:
# Load the datasets
df_teams = pandas.read_csv("dataset/teams.csv")
df_teams_post = pandas.read_csv("dataset/teams_post.csv")
df_series_post = pandas.read_csv("dataset/series_post.csv")
df_players = pandas.read_csv("dataset/players.csv")
df_players_teams = pandas.read_csv("dataset/players_teams.csv")
df_coaches = pandas.read_csv("dataset/coaches.csv")
df_awards_players = pandas.read_csv("dataset/awards_players.csv")

dfs = [df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players]
dfs_names = ["teams", "teams_post", "series_post", "players", "players_teams", "coaches", "awards_players"]

In [4]:
for i in range(len(dfs)):
    print(
        f"{dfs_names[i]}:\nRows: {dfs[i].shape[0]} \tColumns: {dfs[i].shape[1]}\n")

teams:
Rows: 142 	Columns: 61

teams_post:
Rows: 80 	Columns: 5

series_post:
Rows: 70 	Columns: 9

players:
Rows: 893 	Columns: 10

players_teams:
Rows: 1876 	Columns: 43

coaches:
Rows: 162 	Columns: 9

awards_players:
Rows: 95 	Columns: 4



In [7]:
df_series_post.head()


,year,round,series,tmIDWinner,lgIDWinner,tmIDLoser,lgIDLoser,W,L
0,1,FR,A,CLE,WNBA,ORL,WNBA,2,1
1,1,FR,B,NYL,WNBA,WAS,WNBA,2,0
2,1,FR,C,LAS,WNBA,PHO,WNBA,2,0
3,1,FR,D,HOU,WNBA,SAC,WNBA,2,0
4,1,CF,E,HOU,WNBA,LAS,WNBA,2,0
